## Preambule

In [1]:
import warnings
from importlib import reload

import numpy as np
import pandas as pd
import scipy
import xarray as xr
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

# Sobol analysis
# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from SALib.analyze import sobol
from SALib.sample import saltelli
from SALib.test_functions import Ishigami

## Read stuff

In [2]:
path_data = "K:/Data/Data_EffortSharing/DataUpdate_ongoing/startyear_2021/"

In [3]:
xr_dataread = xr.open_dataset(path_data + "xr_dataread.nc")
xr_sobol = xr.open_dataset(path_data + "xr_sobol_GHG_incl.nc").sel(Time=np.arange(2030, 2101, 1))
all_countries_iso = np.load(path_data + "../all_countries.npy", allow_pickle=True)
all_countries_name = np.load(path_data + "../all_countries_names.npy", allow_pickle=True)
all_regions_iso = np.load(path_data + "../all_regions.npy", allow_pickle=True)

## Process stuff

In [4]:
dim_unc_globe = ["Temperature", "Timing", "NegEmis", "NonCO2red"]
dim_unc_fairness = ["variable", "Convergence_year", "Historical_startyear", "Discount_factor"]
dim_unc_choices = ["Scenario", "Risk"]

vals = np.zeros(shape=(3, len(xr_sobol.Time), len(all_countries_iso)))
for dim_i, dim in enumerate([dim_unc_globe, dim_unc_fairness, dim_unc_choices]):
    vals[dim_i] = np.array(
        xr_sobol.sel(Factor=dim, Region=all_countries_iso).sum(dim="Factor").to_array()
    )[0]

In [ ]:
for factor in np.array(xr_sobol.Factor):
    print(factor, float(xr_sobol.sel(Time=2040, Region="USA", Factor=factor).Sobol_index))

factors = [
    "variable",
    "Discount_factor",
    "Historical_startyear",
    "Convergence_year",
    "Risk",
    "Scenario",
    "Temperature",
    "NonCO2red",
    "Timing",
    "NegEmis",
]
factor_names = [
    "Allocation rule",
    "Discount factor",
    "Historical startyear",
    "Convergence year",
    "Climate sensitivity",
    "Socio-economic scenario",
    "Peak temperature",
    "Non-CO<sub>2</sub> reduction",
    "Timing of action",
    "Negative emissions",
]

# factors = ['variable', 'Convergence_year', 'NegEmis', 'NonCO2red',
#             'Temperature', 'Risk', 'Timing', 'Scenario', 'Discount_factor',
#             'Historical_startyear']

# factor_names = ['Allocation rule', 'Convergence year', 'Negative emissions', 'Non-CO<sub>2</sub> reduction',
#                 'Peak temperature', 'Climate sensitivity', 'Timing of action', 'Socio-economic scenario', 'Discount factor', 'Historical startyear',
#                 ]

factor_colors = [
    "goldenrod",
    "salmon",
    "tomato",
    "darkred",
    "dodgerblue",
    "steelblue",
    "forestgreen",
    "lightgreen",
    "yellowgreen",
    "lawngreen",
]

variable 0.5522414105644059
Convergence_year 0.24049700057174322
NegEmis 0.00010847189346472821
NonCO2red 0.0001360817652363187
Temperature 0.036172867348144046
Risk 0.006432116195894081
Timing 0.0002834933047473193
Scenario 0.00042985503574043935
Discount_factor 0.09736505100022648
Historical_startyear 0.06633365232039752


In [ ]:
vals.shape

(3, 71, 197)

In [7]:
mins = np.nanmin(vals, axis=2)
maxs = np.nanmin(vals, axis=2)
mins = (np.array(mins) * 100).round(0).astype(int).astype(str)
maxs = (np.array(maxs) * 100).round(0).astype(int).astype(str)

In [8]:
regions_highlight = ["USA", "CHN", "IND", "BRA", "NGA", "ZAF", "MEX"]
df_regions = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")
continents = []
bubblecolors = []
texts = []
cmap = plt.cm.PuRd
for cty_i, cty in enumerate(all_countries_iso):
    if cty == "USA":
        col = cmap(0.6)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("navy")
        texts.append(cty)
    elif cty == "IND":
        col = cmap(0.7)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("orange")
        texts.append(cty)
    elif cty == "BRA":
        col = cmap(0.8)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("green")
        texts.append(cty)
    elif cty == "CHN":
        col = cmap(0.9)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("red")
        texts.append(cty)
    elif cty == "MEX":
        col = cmap(0.9)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("dodgerblue")
        texts.append(cty)
    elif cty == "NGA":
        col = cmap(0.99)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("purple")
        texts.append(cty)
    elif cty == "ZAF":
        col = cmap(0.99)
        # bubblecolors.append('rgba({},{},{}, {})'.format(int(col[0]*255), int(col[1]*255), int(col[2]*255), 1))
        bubblecolors.append("chocolate")
        texts.append(cty)
    else:
        bubblecolors.append("silver")
        texts.append("")
continents = np.array(continents)
bubblecolors = np.array(bubblecolors)
texts = np.array(texts)

pops = np.array(xr_dataread.Population.sel(Region=all_countries_iso, Time=2020, Scenario="SSP2"))
pops[np.isnan(pops)] = 1
pops_n = pops / np.nanmax(pops)

In [9]:
def clean_data(data_in):
    """
    Cleans data in a format which can be conveniently
    used for drawing traces. Takes a dictionary as the
    input, and returns a list in the following format:

    input = {'key': ['a b c']}
    output = [key, [a, b, c]]
    """
    key = list(data_in.keys())[0]
    data_out = [key]
    for i in data_in[key]:
        data_out.append(list(map(float, i.split(" "))))

    return data_out


conts = []
for i in np.arange(35, 51, 5):
    conts.append(
        {
            str(i): [
                str(i) + " " + str(i) + " " + str(100 - 2 * i),
                str(i) + " " + str(100 - 2 * i) + " " + str(i),
                str(100 - 2 * i) + " " + str(i) + " " + str(i),
            ]
        }
    )
for i in np.arange(55, 101, 5):
    i0 = str(i)
    i1 = str(100 - i)
    conts.append(
        {
            str(i): [
                i0 + " " + i1 + " 0",
                i0 + " 0 " + i1,
                i1 + " 0 " + i0,
                "0 " + i1 + " " + i0,
                "0 " + i0 + " " + i1,
                i1 + " " + i0 + " 0",
                i0 + " " + i1 + " 0",
            ]
        }
    )
conts2 = []
for i in np.arange(35, 51, 5):
    conts2.append(
        {
            str(i): [
                str(i) + " " + str(i) + " " + str(100 - 2 * i),
                str(i) + " " + str(100 - 2 * i) + " " + str(i),
                str(100 - 2 * i) + " " + str(i) + " " + str(i),
            ]
        }
    )
for x in np.arange(55, 101, 5):
    conts2.append(
        {
            str(i): [
                str(x) + " " + "0" + " " + str(100 - x),
                str(x - 5) + " " + "0" + " " + str(100 - (x - 5)),
                str(x - 5) + " " + str(100 - (x - 5)) + " " + "0",
                str(x) + " " + str(100 - x) + " " + "0",
                str(x) + " " + "0" + " " + str(100 - x),
            ]
        }
    )
    conts2.append(
        {
            str(i): [
                "0" + " " + str(100 - x) + " " + str(x),
                "0" + " " + str(100 - (x - 5)) + " " + str(x - 5),
                str(100 - (x - 5)) + " " + "0" + " " + str(x - 5),
                str(100 - x) + " " + "0" + " " + str(x),
                "0" + " " + str(100 - x) + " " + str(x),
            ]
        }
    )
    conts2.append(
        {
            str(i): [
                str(100 - x) + " " + str(x) + " " + "0",
                str(100 - (x - 5)) + " " + str(x - 5) + " " + "0",
                "0" + " " + str(x - 5) + " " + str(100 - (x - 5)),
                "0" + " " + str(x) + " " + str(100 - x),
                str(100 - x) + " " + str(x) + " " + "0",
            ]
        }
    )
bar = px.colors.sequential.Greys  # px.colors.diverging.RdBu_r
# bar = bar[2:-2]
# bar = bar[int(len(bar)/2)-2:]
colors = []
for i in range(len(bar) - 1):
    colors = colors + n_colors(bar[i], bar[i + 1], int(200 / len(bar)), colortype="rgb")
colors = colors + n_colors(bar[i + 1], bar[i + 1], int(200 / len(bar)), colortype="rgb")
conttraces = []
n = 0
for raw_data in conts2:
    data = clean_data(raw_data)

    a = [inner_data[0] for inner_data in data[1:]]
    a.append(data[1][0])  # Closing the loop

    b = [inner_data[1] for inner_data in data[1:]]
    b.append(data[1][1])  # Closing the loop

    c = [inner_data[2] for inner_data in data[1:]]
    c.append(data[1][2])  # Closing the loop

    conttraces.append(
        go.Scatterternary(
            text=data[0],
            a=a,
            b=b,
            c=c,
            mode="lines",
            line=dict(color="black", width=0),
            fill="toself",
            showlegend=False,
            hoverinfo="skip",
            opacity=0.2,
            fillcolor=colors[int(200 / len(conts2) * n)],  # colors_iterator.__next__()
        )
    )
    n += 1
conttraces = conttraces[::-1]

## Plot

In [10]:
def draw_line(fig, name, x1, x2, y1, y2, color):
    fig.add_shape(
        type="line",
        xref="paper",
        yref="paper",
        x0=x1,
        y0=y1,
        x1=x2,
        y1=y2,
        line=dict(color=color, width=3),
    )

In [12]:
fig = make_subplots(
    rows=4,
    cols=3,
    specs=[
        [{"type": "scatterternary", "colspan": 2, "rowspan": 4}, {}, {}],
        [{}, {}, {}],
        [{}, {}, {}],
        [{}, {}, {}],
    ],
    horizontal_spacing=0,
    vertical_spacing=0.05,
)

for n in range(len(conttraces)):
    fig.add_trace(conttraces[n], 1, 1)

# Panel a
for i in np.arange(0.0, 1.01, 0.25):
    args = dict(
        mode="lines",
        showlegend=False,
        hoverinfo="skip",
        marker={
            "size": 15,
            "color": "black",
            "opacity": 1,  # 0.9-i*0.03,
            "line": dict(width=0.0, color="silver"),
        },
        line={
            "width": 0.33,  # np.mean(s[i:i+2])*0.5,
            "color": "black",
            "dash": "dot",
        },
        textfont=dict(size=5, color="black"),
    )
    fig.add_trace(go.Scatterternary(a=[i, i], b=[0, 1 - i], c=[1 - i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(b=[i, i], a=[0, 1 - i], c=[1 - i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(c=[i, i], b=[0, 1 - i], a=[1 - i, 0], **args), 1, 1)

wh_t0 = np.where(np.array(xr_sobol.Time) == 2030)[0][0]
for cty_i, cty in enumerate(all_countries_iso):
    fig.add_trace(
        go.Scatterternary(
            a=vals[0, :, cty_i],
            b=vals[1, :, cty_i],
            c=vals[2, :, cty_i],
            name=cty,
            mode="lines+text",
            showlegend=False,
            text=[""] * (len(vals[0])),
            marker={
                "size": 6,
                "color": bubblecolors[cty_i],
                "opacity": 0.9,  # 0.9-i*0.03,
                "line": dict(width=0, color="black"),
            },
            line={
                "width": 0.25,  # np.mean(s[i:i+2])*0.5,
                "color": "grey",
            },
            textfont=dict(size=(pops_n[cty_i]) ** 0.4 * 25 + 1, color="black"),
        ),
        1,
        1,
    )
    # fig.add_trace(go.Scatterternary(a=vals[0, wh_t0:(wh_t0+5), cty_i],
    #                                 b=vals[1, wh_t0:(wh_t0+5), cty_i],
    #                                 c=vals[2, wh_t0:(wh_t0+5), cty_i],
    #                                 name=cty,
    #                                                 mode='lines+text',
    #                                                 showlegend=False,
    #                                                 text=['']*(len(vals[0])),
    #                                                 marker={'size': 6,
    #                                                         'color': bubblecolors[cty_i],
    #                                                         'opacity': 0.9,#0.9-i*0.03,
    #                                                         'line' :dict(width=0, color='black')},
    #                                                 line={'width': 0.25, #np.mean(s[i:i+2])*0.5,
    #                                                         'color': 'grey'},
    #                                                 textfont=dict(size=(pops_n[cty_i])**0.4*25+1,
    #                                                         color='black')), 1, 1)
for ci, cty in enumerate(regions_highlight):
    cty_i = np.where(all_countries_iso == cty)[0][0]
    fig.add_trace(
        go.Scatterternary(
            a=vals[0, wh_t0 : (wh_t0 + 71), cty_i],
            b=vals[1, wh_t0 : (wh_t0 + 71), cty_i],
            c=vals[2, wh_t0 : (wh_t0 + 71), cty_i],
            name=cty,
            mode="lines+markers",
            showlegend=False,
            text=[
                "2030",
                "",
                "",
                "",
                "",
                "",
                "'60",
                "",
                "",
                "",
                "'80",
                "",
                "",
                "",
                "2100",
            ],  # ["'., str(i-2000) for i in [2040, 2050, 2060, 2070, 2080, 2090]],
            marker={
                "size": [8] * 79,  # ]*(len(vals[0])), #[(pops_n[cty_i])**0.4*60+4]+
                "color": bubblecolors[cty_i],
                "opacity": 0.9,  # 0.9-i*0.03,
                "line": dict(width=0.5, color="white"),
            },
            line={
                "width": 3,  # np.mean(s[i:i+2])*0.5,
                "color": bubblecolors[cty_i],
            },
            textposition="top center",
            textfont=dict(size=16, color=bubblecolors[cty_i]),
        ),
        1,
        1,
    )

    for t_i, t in enumerate([2100, 2030, 2050, 2100]):
        wh_t = np.where(np.array(xr_sobol.Time) == t)[0][0]
        fig.add_trace(
            go.Scatterternary(
                a=[vals[0, wh_t, cty_i]],
                b=[vals[1, wh_t, cty_i]],
                c=[vals[2, wh_t, cty_i]],
                name=cty,
                mode="markers+text",
                showlegend=False,
                text=[["<b>" + cty], [""], [""], ["2100"]][
                    t_i
                ],  # ["'., str(i-2000) for i in [2040, 2050, 2060, 2070, 2080, 2090]],
                marker={
                    "size": [[20], [20], [15], [20]][t_i],
                    #'symbol': 'x',
                    "color": bubblecolors[cty_i],
                    "opacity": 0.9,  # 0.9-i*0.03,
                    "line": dict(width=0.5, color="white"),
                },
                line={
                    "width": 3,  # np.mean(s[i:i+2])*0.5,
                    "color": bubblecolors[cty_i],
                },
                textposition=[
                    [
                        "middle right",
                        "middle right",
                        "middle left",
                        "middle right",
                        "top center",
                        "middle left",
                        "middle right",
                    ][ci],
                    "middle right",
                    "middle right",
                    [
                        "bottom center",
                        "bottom right",
                        "bottom center",
                        "bottom center",
                        "bottom center",
                        "bottom center",
                        "bottom center",
                    ][ci],
                ][t_i],
                textfont=dict(size=[18, 16, 16, 16, 16, 16][t_i], color=bubblecolors[cty_i]),
            ),
            1,
            1,
        )
fig.update_layout(
    {
        "plot_bgcolor": "rgb(243, 243, 243)",
        "ternary": {
            "sum": 1,
            "bgcolor": "white",
            # "paper_bgcolor":'rgba(0,0,0,0)',
            # "plot_bgcolor":'rgba(0,0,0,0)',
            "aaxis": {
                "title": "",
                "min": 0.0,
                "titlefont": {"size": 25},  # Climate target<br>
                "linewidth": 0,
                "ticks": "outside",
                "tickmode": "array",
                "tickvals": np.arange(0, 1.01, 0.25),
                "ticktext": [str(int(i * 100)) + "%" for i in np.arange(0, 1.01, 0.25)],
                "tickfont": {"size": 20},
            },
            "baxis": {
                "title": "",
                "min": 0,
                "titlefont": {"size": 25},  # &nbsp; &nbsp; &nbsp;#Model &nbsp; &nbsp;
                "linewidth": 2,
                "ticks": "outside",
                "tickmode": "array",
                "tickvals": np.arange(0, 1.01, 0.25),
                "ticktext": [str(int(i * 100)) + "%" for i in np.arange(0, 1.01, 0.25)],
                "tickangle": 60,
                "tickfont": {"size": 20},
            },
            "caxis": {
                "title": "",
                "min": 0,
                "titlefont": {"size": 25},  # Other<br>scenario assumptions
                "linewidth": 2,
                "ticks": "outside",
                "tickmode": "array",
                "tickvals": np.arange(0, 1.01, 0.25),
                "ticktext": [str(int(i * 100)) + "%" for i in np.arange(0, 1.01, 0.25)],
                "tickangle": -60,
                "tickfont": {"size": 20},
            },
        },
    }
)

# Panel b-d
for cty_i, cty in enumerate(["USA", "CHN", "IND", "NGA"]):
    for factor_i, factor in enumerate(factors):
        col = factor_colors[factor_i]
        val_i = xr_sobol.sel(Region=cty, Factor=factor).Sobol_index
        if factor_i == 0:
            fill = "tozeroy"
        else:
            fill = "tonexty"
        fig.add_trace(
            go.Scatter(
                x=xr_sobol.Time,
                y=val_i,
                mode="lines",
                marker={"color": col},
                fill=fill,
                name=factor_names[factor_i],
                legend=[
                    "legend1",
                    "legend1",
                    "legend1",
                    "legend1",
                    "legend2",
                    "legend2",
                    "legend3",
                    "legend3",
                    "legend3",
                    "legend3",
                ][factor_i],
                showlegend=[True, False, False, False, False, False, False][cty_i],
                stackgroup="one",
            ),
            range(1, 6)[cty_i],
            3,
        )

fig.update_geos(
    visible=False,
    showlakes=True,
    lakecolor="rgb(255, 255, 255)",
    projection_type="natural earth",
    resolution=110,
    showcoastlines=False,
)

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    # template='ggplot2',
    height=1100,
    width=1600,
)
fig.update_xaxes(visible=False, row=1, col=3)
fig.update_xaxes(visible=False, row=2, col=3)
fig.update_xaxes(visible=False, row=3, col=3)

args = dict(xref="paper", yref="paper", showarrow=False)
annotations = [
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=24),
        y=0.98,
        x=0.0,
        text="<b>(a) Relative impact of<br>the three dimensions</b>",
    ),
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=17),
        y=0.88,
        x=0.0,
        text="Expressed in fraction of variance explained",
    ),
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=24),
        y=1.04,
        x=0.85,
        text="<b>(b) United States</b>",
    ),
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=24),
        y=0.78,
        x=0.775,
        text="<b>(c) China</b>",
    ),
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=24),
        y=0.49,
        x=0.775,
        text="<b>(d) India</b>",
    ),
    dict(
        xref="paper",
        yref="paper",
        ax=0,
        align="left",
        ay=-0,
        showarrow=False,
        font=dict(color="black", size=24),
        y=0.21,
        x=0.79,
        text="<b>(e) Nigeria</b>",
    ),
    dict(
        text="Global strategies dominate",
        x=0.215,
        y=1.12,
        **args,
        font=dict(color="black", size=25),
    ),
    dict(
        text="temperature, negative emissions, timing, non-CO<sub>2",
        x=0.19,
        y=1.08,
        **args,
        font=dict(color="black", size=16),
    ),
    dict(text="Equity dominates", x=-0.05, y=0.19, **args, font=dict(color="black", size=25)),
    dict(
        text="allocation rule, discounting,<br>historical startyear<br>convergence year",
        x=-0.05,
        y=0.11,
        **args,
        font=dict(color="black", size=16),
    ),
    dict(
        text="Physical and social<br>uncertainties<br>dominate",
        x=0.54,
        y=0.08,
        **args,
        font=dict(color="black", size=25),
    ),
    dict(
        text="socio-economic scenario,<br>climate sensitivity",
        x=0.54,
        y=0.02,
        **args,
        font=dict(color="black", size=16),
    ),
]
for ann in annotations:
    fig.add_annotation(**ann)

fig.update_layout(
    legend1=dict(
        x=0.2,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="v",
        title="Equity considerations",
        font=dict(size=20),
    ),
    legend2=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="v",
        title="Scientific uncertainties",
        font=dict(size=20),
    ),
    legend3=dict(
        x=0.8,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        title="Global discussions",
        orientation="v",
        font=dict(size=20),
    ),
    paper_bgcolor="white",  # Set the background color
    plot_bgcolor="white",
    font=dict(size=20, color="black"),
)
# change ytick labels in row=1, col=3 panel
fig.update_yaxes(tickvals=[0, 0.5, 1], ticktext=["0%", "50%", "100%"], row=1, col=3)
fig.update_yaxes(tickvals=[0, 0.5, 1], ticktext=["0%", "50%", "100%"], row=2, col=3)
fig.update_yaxes(tickvals=[0, 0.5, 1], ticktext=["0%", "50%", "100%"], row=3, col=3)
fig.update_yaxes(tickvals=[0, 0.5, 1], ticktext=["0%", "50%", "100%"], row=4, col=3)
fig.show()
fig.write_image("K:/Code/effort-sharing/Figures/Paper_FairShares/V3/Figure_2.png", scale=5)
fig.write_image("K:/Code/effort-sharing/Figures/Paper_FairShares/V3/SVG/Figure_2.svg", scale=5)